In [1]:
# python -m pip install oracledb --upgrade

In [2]:
import warnings
warnings.filterwarnings("ignore")

import sys
import os
import numpy as np
import pandas as pd
import seaborn as sns
import oracledb
import configparser
from IPython.display import display

In [3]:
host = r'localhost'
port = 51521
service_name = r'ORCLPDB1.localdomain'
username = r'test'
password = r'test'

In [4]:
class Oracle(object):
    
    def __init__(self, username, password, hostname, port, servicename):
        
        self.hostname = hostname
        self.port = port
        self.servicename = servicename
        self.username = username
        self.__password = password
    
    def connect(self):
        """ Connect to the database. """
        
        connection = None
    
        try:
            print('Database connecting ..')
            params = oracledb.ConnectParams(host=self.hostname, port=self.port, service_name=self.servicename)
            connection = oracledb.connect(user=self.username, password=self.__password, params=params)
        except oracledb.DatabaseError as error:
            # Log error as appropriate
            print('Whoops!')
            print(error)
        else:
            print('Database version:', connection.version)
            self.connection = connection

    def disconnect(self):
        """
        Disconnect from the database. If this fails, for instance
        if the connection instance doesn't exist, ignore the exception.
        """

        try:
            self.connection.close()
        except oracledb.DatabaseError as error:
            pass
        else:
            print('Database disconnecting.')

    def execute(self, sql, commit = False):
        """
        Execute whatever SQL statements are passed to the method;
        commit if specified. Do not specify fetchall() in here as
        the SQL statement may not be a select.
        bindvars is a dictionary of variables you pass to execute.
        """
        
        conn = self.connection
        cursor = conn.cursor()
        
        try:
            cursor.execute(sql)
            
            # Only commit if it-s necessary.
            if commit:
                conn.commit()
                        
        except oracledb.DatabaseError as error:
            if commit:
                conn.rollback()
                
            # Log error as appropriate
            print('Whoops!')
            print(error)
            
        finally:
            cursor.close() 

In [5]:
oracle = Oracle(username, password, host, port, service_name)
oracle.connect()

Database connecting ..
Database version: 12.2.0.1.0


In [6]:
sql = """
SELECT *
FROM
    german_credit_data
FETCH FIRST 10 ROWS ONLY
"""

In [7]:
df = pd.read_sql(sql, con=oracle.connection)
display(df)

,ID,AGE,SEX,JOB,HOUSING,SAVING_ACCOUNTS,CHECKING_ACCOUNT,CREDIT_AMOUNT,DURATION,PURPOSE
0,537,37,female,2,own,little,moderate,3612,18,furniture/equipment
1,538,42,male,3,free,little,little,7763,48,car
2,539,45,female,1,own,little,rich,3049,18,furniture/equipment
3,540,23,male,2,rent,little,moderate,1534,12,radio/TV
4,541,60,male,2,free,little,NA,2032,24,car
5,542,31,male,2,own,NA,little,6350,30,furniture/equipment
6,543,34,male,1,own,little,rich,2864,18,furniture/equipment
7,544,61,male,1,own,little,NA,1255,12,car
8,545,43,male,2,free,little,little,1333,24,car
9,546,37,female,2,own,little,NA,2022,24,car


In [8]:
table = "credit_data"

col = df.columns.to_list()
columns = ", ".join(col)
le = len(col)

data = list(df.itertuples(index=False, name=None))

stmt = "INSERT INTO "+table+" ("+columns+") VALUES "+str(data[0])
print(stmt)

INSERT INTO credit_data (ID, AGE, SEX, JOB, HOUSING, SAVING_ACCOUNTS, CHECKING_ACCOUNT, CREDIT_AMOUNT, DURATION, PURPOSE) VALUES (537, 37, 'female', 2, 'own', 'little', 'moderate', 3612, 18, 'furniture/equipment')


In [9]:
oracle.execute(stmt, True)

In [10]:
sql = """
SELECT *
FROM
    credit_data
"""

test = pd.read_sql(sql, con=oracle.connection)
display(test)

,ID,AGE,SEX,JOB,HOUSING,SAVING_ACCOUNTS,CHECKING_ACCOUNT,CREDIT_AMOUNT,DURATION,PURPOSE
0,537,37,female,2,own,little,moderate,3612,18,furniture/equipment


In [11]:
table = "credit_data"

col = df.columns.to_list()
columns = ", ".join(col)
le = len(col)

for index, row in df.iterrows():
    data = tuple(row.to_list())
    stmt = "INSERT INTO "+table+" ("+columns+") VALUES "+str(data)
    oracle.execute(stmt, True)

In [12]:
sql = """
SELECT *
FROM
    credit_data
"""

test = pd.read_sql(sql, con=oracle.connection)
display(test)

,ID,AGE,SEX,JOB,HOUSING,SAVING_ACCOUNTS,CHECKING_ACCOUNT,CREDIT_AMOUNT,DURATION,PURPOSE
0,537,37,female,2,own,little,moderate,3612,18,furniture/equipment
1,537,37,female,2,own,little,moderate,3612,18,furniture/equipment
2,538,42,male,3,free,little,little,7763,48,car
3,539,45,female,1,own,little,rich,3049,18,furniture/equipment
4,540,23,male,2,rent,little,moderate,1534,12,radio/TV
5,541,60,male,2,free,little,NA,2032,24,car
6,542,31,male,2,own,NA,little,6350,30,furniture/equipment
7,543,34,male,1,own,little,rich,2864,18,furniture/equipment
8,544,61,male,1,own,little,NA,1255,12,car
9,545,43,male,2,free,little,little,1333,24,car


In [13]:
oracle.disconnect()

Database disconnecting.
